#### --- RAW DATA ANALYSIS ---

In [1]:
#import libraries
from pyspark.sql import SparkSession

spark = SparkSession.\
    builder.\
    appName('raw_data_cap_project').\
    master('yarn').\
    getOrCreate()

In [2]:
raw_df = spark\
.read\
.format('csv')\
.option("InferSchema","True")\
.option("header","True")\
.load('/public/trendytech/datasets/accepted_2007_to_2018Q4.csv')

In [3]:
# convert df to table for SQL queries
raw_df.createOrReplaceTempView("lending_club_data")

In [5]:
# viewing the top5 rows of dataset
spark.sql("select * from lending_club_data LIMIT 5")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
68407277,null,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,null,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723916800001,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,null,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,null,null,null,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,null,null,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Cash,N,null,null,null,null,null,null
68355089,null,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,null,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.66,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,null,Mar-2019,699.0,695.0,0.0,null,1.0,Individual,null,null,null,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,null,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,null,null,null,null,null,null,null,null

- The id column in the dataset represents the unique identifier for each loan (loan_id).

- The member_id column contains only null values, so it cannot be used to identify borrowers.

- Since a single member can take multiple loans, the same member might appear more than once in the dataset.

- To uniquely identify each member, we use a combination of features that are likely to remain consistent across multiple loans taken by the same person.

- We selected the following nine features to construct a unique member ID: ["emp_title", "emp_length", "home_ownership", "annual_inc", "zip_code", "addr_state", "grade", "sub_grade", "verification_status"].

In [4]:
# Import necessary PySpark functions
from pyspark.sql.functions import sha2,concat_ws

# Create a new column 'name_sha2' by combining selected features and applying SHA-256 hash.
# This hashed value serves as a unique member ID.
new_df = raw_df.withColumn("name_sha2", sha2(concat_ws("||", *["emp_title", "emp_length", "home_ownership", "annual_inc", "zip_code", "addr_state", "grade", "sub_grade","verification_status"]), 256))

In [5]:
# Register the DataFrame as a temporary SQL view for querying
new_df.createOrReplaceTempView("newtable")

#### --- VALIDATION STEPS ---

In [11]:
# checking total no of records

row_count = spark.sql("select count(*) as total from lending_club_data").collect()[0]['total']
print(f'No of rows in dataframe: {row_count}')

No of rows in dataframe: 2260701


In [9]:
# Number of columns in the dataframe
len(raw_df.columns)

151

In [12]:
# checking number of unique member IDs (name_sha2 values)
spark.sql("select count(distinct(name_sha2)) from newtable")

count(DISTINCT name_sha2)
2257384


In [13]:
# Find how many records share the same hashed ID (i.e., potential duplicates)
spark.sql("""
    SELECT name_sha2, COUNT(*) AS total_cnt 
    FROM newtable 
    GROUP BY name_sha2 
    HAVING total_cnt > 1 
    ORDER BY total_cnt DESC
""")

name_sha2,total_cnt
e3b0c44298fc1c149...,33
e4c167053d5418230...,5
3f87585a20f702838...,4
76b577467eda5bdbc...,4
ad8e5d384dae17e06...,4
819453be77718d747...,3
059d401bb603d9a80...,3
d93573e2883e37904...,3
53789bea7edc660ed...,3
291ca1b09ef11ca3e...,3


record1 has 33 common hash id which is exception/ outlier 

In [14]:
# Investigate empty or default hash values (often generated from null or empty feature combinations)
spark.sql("select * from newtable where name_sha2 like 'e3b0c44298fc1c149%'")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,name_sha2
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c44298fc1c149...
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c

#### --- EXPORT CUSTOMERS DATA ---

In [38]:
# Export customer-specific attributes with the hashed member_id and additional country field as customers_data in a csv file
spark.sql("""select name_sha2 as member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,'USA' as country,grade,sub_grade,
verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint from newtable
""").repartition(1).write \
.option("header","true")\
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv017499/lendingclubproject/raw/customers_data_csv") \
.save()

In [41]:
# Load the saved customers data for inspection
customers_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/user/itv017499/lendingclubproject/raw/customers_data_csv")

In [42]:
# View the loaded DataFrame
customers_df.show(5)

+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_inc|addr_state|zip_code|country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|annual_inc_joint|verification_status_joint|
+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|b59d80da191f5b573...|                null|      null|          RENT|   50000.0|        OR|   973xx|    USA|    A|       A5|    Source Verified|         8600.0|      Individual|            null|                     null|
|202d9f56ecb7c3bc9...|      police officer|   7 years|           OWN|   85000.0|        TX|   799xx|    USA|    A|  

#### --- EXPORT LOANS DATA ---

In [43]:
# Extract and save loan-related data with the member_id and loan_id
spark.sql("""select id as loan_id, name_sha2 as member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,
title from newtable""").repartition(1).write \
.option("header",True)\
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv017499/lendingclubproject/raw/loans_data_csv") \
.save()

In [44]:
# Load the loan data
loans_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/user/itv017499/lendingclubproject/raw/loans_data_csv")

In [45]:
# Load the loan data
loans_df.show(5)

+--------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+------------------+--------------------+
| loan_id|           member_id|loan_amnt|funded_amnt|     term|int_rate|installment| issue_d|loan_status|           purpose|               title|
+--------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+------------------+--------------------+
|14408468|5d7676571dee53d06...|   5000.0|     5000.0|36 months|   14.16|     171.28|Apr-2014| Fully Paid|debt_consolidation|  Debt consolidation|
|14520567|31fd1ef036c5caf26...|  11000.0|    11000.0|36 months|    9.67|     353.24|Apr-2014| Fully Paid|             other|               Other|
|14708730|9b48253ca5848fa78...|  25000.0|    25000.0|36 months|   11.99|     830.24|May-2014|Charged Off|       credit_card|Credit card refin...|
|14491003|d6b8f2e32be148721...|   1500.0|     1500.0|36 months|   23.43|      58.41|Apr-2014| Fully Paid|  renewable_energy|

#### --- EXPORT LOAN repayments DATA ---

In [46]:
# Extract and export repayment details related to each loan

spark.sql("""select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d from newtable""").repartition(1).write \
.option("header",True)\
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv017499/lendingclubproject/raw/loans_repayments_csv") \
.save()

In [47]:
# Load the repayment data
loans_repayments_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/user/itv017499/lendingclubproject/raw/loans_repayments_csv")

In [48]:
# View the repayments DataFrame
loans_repayments_df.show(5)

+--------+---------------+-------------+------------------+------------------+---------------+------------+------------+
| loan_id|total_rec_prncp|total_rec_int|total_rec_late_fee|       total_pymnt|last_pymnt_amnt|last_pymnt_d|next_pymnt_d|
+--------+---------------+-------------+------------------+------------------+---------------+------------+------------+
|68407277|         3600.0|       821.72|               0.0| 4421.723916800001|         122.67|    Jan-2019|        null|
|68355089|        24700.0|       979.66|               0.0|          25679.66|         926.35|    Jun-2016|        null|
|68341763|        20000.0|      2705.92|               0.0|22705.924293878397|        15813.3|    Jun-2017|        null|
|66310712|       19102.35|     12361.66|               0.0|          31464.01|          829.9|    Feb-2019|    Apr-2019|
|68476807|        10400.0|       1340.5|               0.0|           11740.5|       10128.96|    Jul-2016|        null|
+--------+---------------+------

#### --- EXPORT LOAN DEFAULTERS DATA ---

In [6]:
# Export credit behavior and delinquency information for identifying defaulters
spark.sql("""select name_sha2 as member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record from newtable""").repartition(1).write \
.option("header",True)\
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv017499/lendingclubproject/raw/loans_defaulters_csv") \
.save()

In [7]:
# Load the defaulters data

loans_defaulters_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/user/itv017499/lendingclubproject/raw/loans_defaulters_csv")

In [8]:
# View the defaulters DataFrame
loans_defaulters_df.show(5)

+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|           member_id|delinq_2yrs|delinq_amnt|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|
+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|707271898dcabc8b2...|        0.0|        0.0|    1.0|                 1.0|           0.0|               0.0|                  null|                  83.0|
|8e1ea10aca3c4ad8f...|        1.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  12.0|                  null|
|1d6546a2cbc1fd240...|        2.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  20.0|                  null|
|d6208beced388988f...|        0.0|        0.0|    0.0|          

In [10]:
spark.stop()